# Course Project - Convex Optimization for Signal Processing and Communications

- Student: Lucas von Ancken Garcia
- Matrikelnummer: 2576600 

## Problem 1

Given a single-user (point-to-point) MIMO communication channel with the channel matrix $H$ and zero mean
colored noise $n$ at the receiver antennas, the received symbol at the receiver is $y = H^H \cdot x + n$, where $x$ is the
transmitted symbol with zero mean. We want to **design the optimal transmit covariance matrix** that **maximizes the transmission rate** of the MIMO system. We assume that some of the **antennas have power constraints** that
restrict the transmitted power. Furthermore, **a sum power constraint applies**.

Generate required data to implement the problem formulation in CVX. Average the simulation results with Monte Carlo iterations.

In [55]:
import numpy as np
import cvxpy as cvx
from scipy import signal


np.random.seed(1)
np.set_printoptions(precision=3, suppress=True)

In [56]:
# Generation of the complex Channel Matrix H according to Rayleigh Fading Model
def generate_rayleigh_channel_matrix(m, n):
    real_part = np.random.normal(0, np.sqrt(0.5), (m,n))
    imag_part = np.random.normal(0, np.sqrt(0.5), (m,n))

    return real_part + 1j*imag_part

def generate_white_noise(n):
    real_part = np.random.normal(0, np.sqrt(0.5), n)
    imag_part = np.random.normal(0, np.sqrt(0.5), n)

    return real_part + 1j*imag_part

def generate_random_channel_vector(n, mean, std):
    real_part = np.random.normal(mean, std, n)
    imag_part = np.random.normal(mean, std, n)
    
    return (real_part + 1j*imag_part).reshape(n,1)

In [57]:
def design_cov_matrix(H, Rn, power_limit_per_tx, total_power):
    num_tx, num_rx  = H.shape
    Rn_inv = np.linalg.inv(Rn)

    P = cvx.Variable((num_tx, num_tx))

    objective = cvx.Maximize(cvx.log_det(np.identity(num_rx) + Rn_inv @ H.conjugate().T @ P @ H))

    constraints = [P >> 0, cvx.trace(P) <= total_power]

    for i in range(num_tx):
        constraints.append(P[i, i] <= power_limit_per_tx[i])

    # Solve the problem
    prob = cvx.Problem(objective, constraints)
    prob.solve()

    return (prob.value, P.value)

In [58]:
num_tx = 8  # Number of transmit antennas
num_rx = 5  # Number of receive antennas

power_noise = 2
total_power = 90

power_limit_per_tx = [15, 20, 10, 8, 15, 18, 9, 14]

H = cvx.Parameter((num_tx, num_rx), complex=True)
Rn_inv = cvx.Parameter((num_rx, num_rx))

P = cvx.Variable((num_tx, num_tx))

objective = cvx.Maximize(cvx.log_det(np.identity(num_rx) + Rn_inv @ cvx.conj(H.T) @ P @ H))

constraints = [P >> 0,  P == P.T, cvx.trace(P) <= total_power]

for i in range(num_tx):
    constraints.append(P[i, i] <= power_limit_per_tx[i])

prob = cvx.Problem(objective, constraints)

H.value = generate_rayleigh_channel_matrix(num_tx, num_rx)
Rn_inv.value = np.identity(num_rx)*(1/power_noise)

prob.solve(verbose=True)

                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) Jun 16 11:58:17 AM: Your problem has 64 variables, 137 constraints, and 65 parameters.
(CVXPY) Jun 16 11:58:17 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 16 11:58:17 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 16 11:58:17 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 16 11:58:17 AM: Compiling problem (target solver=SCS).
(CVXPY) Jun 16 11:58:17 AM: Reduction chain: EvalParams -> Complex2Real -> FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixSt

c:\Users\User\Documents\Selbststudium\TUDa_MachineLearning-DS-Exercises\Convex Optimization\Project\.venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:235: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)


18.170637134826322

In [63]:
prob.dual

AttributeError: 'Problem' object has no attribute 'dual'

In [60]:
print("CONSTANTS")
print("")
print("Number of Transmitters: ", num_tx)
print("Number of Receivers: ", num_rx)
print("Power Limits per TX: {} (sum = {})".format(power_limit_per_tx, sum(power_limit_per_tx)))
print("Total Power Limit", total_power)
print("")
print("")

print("INPUT PARAMETERS")
print("")
print("Power noise: ", power_noise)
print("Channel Matrix  H = ")
print(H.value)
print("")
print("")

print("RESULTS")
print("")
print("Status:", prob.status)
print("Optimal value: ", objective.value)
print("Num of Constraints: ", len(constraints))
print("Eigenvalues of optimal P = ", np.linalg.eigvalsh(P.value))
print("")
print("--------------------")
print("P* = ")
print(P.value)
print("Actual Total Power of the Optimal Solution: {}".format(P.value.trace()))
print("--------------------")
print("Lagrange Multipliers")
print("For P >= 0: ")
print(constraints[0].dual_value)
print("For total power:")
print(constraints[1].dual_value)
print("For the limit power of each TX:")
print([x.dual_value for x in constraints[2:]])

CONSTANTS

Number of Transmitters:  8
Number of Receivers:  5
Power Limits per TX: [15, 20, 10, 8, 15, 18, 9, 14] (sum = 109)
Total Power Limit 90


INPUT PARAMETERS

Power noise:  2
Channel Matrix  H = 
[[ 1.149-0.136j -0.433-0.628j -0.373-0.528j -0.759+1.197j  0.612+0.036j]
 [-1.627-0.45j   1.234+0.135j -0.538+1.485j  0.226+0.085j -0.176+0.436j]
 [ 1.034+0.212j -1.457-0.249j -0.228-0.808j -0.272-0.247j  0.802-0.148j]
 [-0.778+0.415j -0.122+0.593j -0.621+0.658j  0.03 +0.202j  0.412+0.626j]
 [-0.778-0.533j  0.809+0.886j  0.638+0.363j  0.355-0.211j  0.637+0.345j]
 [-0.483-0.053j -0.087+0.8j   -0.662+1.075j -0.189+1.545j  0.375-0.987j]
 [-0.489-1.021j -0.281-0.357j -0.486+0.113j -0.598+0.62j  -0.475+0.223j]
 [-0.009-1.43j  -0.79 -0.217j  0.166+0.585j  1.174+0.163j  0.525+0.539j]]


RESULTS

Status: optimal
Optimal value:  18.170637134826322
Num of Constraints:  11
Eigenvalues of optimal P =  [ 0.     0.     0.    16.869 16.944 18.004 18.992 19.191]

--------------------
P* = 
[[ 8.602 -4

In [48]:
P_list = [P.value]
H_list = [H.value]
power_noise_list = [power_noise]
optimal_value_list = [prob.value]
for i in range(19):
    H.value = generate_rayleigh_channel_matrix(num_tx, num_rx)

    power_noise = np.random.uniform(low=0, high=5)
    Rn_inv.value = np.identity(num_rx)*(1/power_noise)

    prob.solve()

    H_list.append(H.value)
    power_noise_list.append(power_noise)

    P_list.append(P.value)
    optimal_value_list.append(prob.value)

H_avg = sum(H_list)/len(H_list)
power_noise_avg = sum(power_noise_list)/len(power_noise_list)

P_avg = sum(P_list)/len(P_list)
optimal_value_avg = sum(optimal_value_list)/len(optimal_value_list)

In [37]:
print("AVERAGE INPUT PARAMETERS")
print("")
print("Average power noise = ", power_noise_avg)
print("Average Channel Matrix H = ")
print(H_avg)
print("")
print("")

print("RESULTS")
print("")
print("Optimal Value Average: {}".format(optimal_value_avg))
print("Average P* = ")
print(P_avg)

AVERAGE INPUT PARAMETERS

Average Channel Matrix H = 
[[-0.033+0.171j  0.126-0.176j -0.08 -0.054j -0.   +0.334j -0.318+0.206j]
 [ 0.364-0.179j  0.089-0.102j  0.161+0.074j  0.211-0.188j  0.084+0.126j]
 [ 0.089-0.085j  0.11 +0.044j -0.091+0.079j -0.121+0.098j  0.048-0.05j ]
 [-0.197-0.093j  0.061+0.318j -0.246-0.003j -0.054+0.071j -0.303-0.108j]
 [-0.115+0.164j  0.03 -0.247j -0.204+0.097j -0.253+0.317j  0.021-0.106j]
 [ 0.043-0.075j  0.147+0.016j  0.077+0.179j  0.032+0.171j -0.284-0.079j]
 [-0.026-0.228j  0.326-0.053j  0.322+0.088j  0.385+0.175j  0.289+0.054j]
 [-0.15 -0.034j -0.003-0.181j  0.216-0.014j  0.213-0.086j  0.129+0.286j]]


RESULTS

Optimal Value Average: 18.878338629790836
Average P* = 
[[12.315  0.194 -0.169 -0.542 -0.362 -0.963 -0.444  0.298]
 [ 0.194 14.224 -1.049  0.832  1.512 -0.307 -0.372 -1.143]
 [-0.169 -1.049  9.864 -1.604 -0.406 -0.593  0.649 -1.212]
 [-0.542  0.832 -1.604  7.712 -0.251 -0.018 -0.433  0.651]
 [-0.362  1.512 -0.406 -0.251 13.102 -0.088 -0.364 -1.303]

## Problem 2
We assume further that a single-antenna co-channel user with channel vector hc is now present. The received symbol is $y_c = h^H_c \cdot x + n_c$, where $n_c$ is the zero mean colored noise at this user. We want that this user does not receive any interference in expectation, i.e., $E[∥y_c∥^2] = 0$.

Please modify the problem formulation to adopt the new constraint and solve it in CVX.

In [52]:
hc = generate_random_channel_vector(num_tx, 0, 2)

new_constraints = prob.constraints + [cvx.trace((hc @ hc.conjugate().transpose()) @ P) == 0]

new_prob = cvx.Problem(objective, new_constraints)
new_prob.solve(verbose=True, max_iters=int(2e5))

                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) Jun 16 11:45:38 AM: Your problem has 64 variables, 138 constraints, and 65 parameters.
(CVXPY) Jun 16 11:45:38 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 16 11:45:38 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 16 11:45:38 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 16 11:45:38 AM: Compiling problem (target solver=SCS).
(CVXPY) Jun 16 11:45:38 AM: Reduction chain: EvalParams -> Complex2Real -> FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixSt

15.547624888091848

In [51]:
interference =  abs(np.trace(P.value @ (hc @ hc.transpose().conjugate())))

print("Status:", new_prob.status)
print("New optimal value: ", new_prob.value)
print("New num of Constraints: ", len(new_constraints))
print("--------------------")
print("new P* = ")
print(P.value)
print("Actual Total Power of the new Optimal Solution: {}".format(P.value.trace()))
print("Interference in new user: ", interference)
print("--------------------")
print("New Lagrange Multipliers")
print("For P >= 0: ")
print(new_constraints[0].dual_value)
print("For total power:")
print(new_constraints[1].dual_value)
print("For the limit power of each TX:")
print([x.dual_value for x in new_constraints[2:]])


Status: optimal_inaccurate
New optimal value:  17.141915070950077
New num of Constraints:  12
--------------------
new P* = 
[[ 5.746  2.82   1.664 -3.008 -1.064 -7.569  0.107  2.772]
 [ 2.82  19.34  -0.598  2.728  1.958 -0.524  1.533 -1.162]
 [ 1.664 -0.598 10.     1.712 -0.742  1.365 -1.087  8.779]
 [-3.008  2.728  1.712  8.    -5.518  3.316 -2.415 -1.366]
 [-1.064  1.958 -0.742 -5.518 15.     4.45  -2.39   0.082]
 [-7.569 -0.524  1.365  3.316  4.45  13.746  2.339  0.317]
 [ 0.107  1.533 -1.087 -2.415 -2.39   2.339  9.     1.627]
 [ 2.772 -1.162  8.779 -1.366  0.082  0.317  1.627  9.168]]
Actual Total Power of the new Optimal Solution: 89.99999672542427
Interference in new user:  1.113955051181004e-06
--------------------
New Lagrange Multipliers
For P >= 0: 
[[ 3.848 -0.884 -0.299  1.524  0.549  1.524  0.388 -0.888]
 [-0.884  0.347 -0.246 -0.767 -0.473 -0.039 -0.495  0.526]
 [-0.299 -0.246  0.733  0.784  0.713 -0.798  0.86  -0.657]
 [ 1.524 -0.767  0.784  1.815  1.221 -0.297  1.326 